# Imports

* The objective of this assignments is to build the **Decoder** part of the Transformer architecture.
* We will be using the **PyTorch** framework to implement the following components
  * Decoder Layer that contains
    * Multi-Head Masked Attention (MHMA) Module
    * Multi-Head Cross Attention (MHMA) Module
    * Position-wise Feed Forward Neural Network

  * Implement CLM

* **DO NOT** USE Built-in **TRANSFORMER LAYERS** as it affects the reproducibility.

* You will be given with a configuration file that contains information on various hyperparameters such as embedding dimension, vocabulary size,number heads and so on

* Use ReLU activation function and Stochastic Gradient Descent optimizer
* Here are a list of helpful Pytorch functions (does not mean you have to use all of them) for this subsequent assignments
  * [torch.matmul](https://pytorch.org/docs/stable/generated/torch.matmul.html#torch-matmul)
  * [torch.bmm](https://pytorch.org/docs/stable/generated/torch.bmm.html)
  * torch.swapdims
  * torch.unsqueeze
  * torch.squeeze
  * torch.argmax
  * [torch.Tensor.view](https://pytorch.org/docs/stable/generated/torch.Tensor.view.html)
  * [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)
  * [torch.nn.Parameter](https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html)
  * torch.nn.Linear
  * torch.nn.LayerNorm
  * torch.nn.ModuleList
  * torch.nn.Sequential
  * [torch.nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
  
* Important: Do not set any global seeds.

* Helpful resources to get started with

 * [Andrej Karpathys Nano GPT](https://github.com/karpathy/nanoGPT)
 * [PyTorch Source code of Transformer Layer](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html)



In [ ]:
import torch
from torch import Tensor

import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.nn.functional import one_hot

import torch.optim as optim

from  pprint import pprint
from yaml import safe_load
import copy
import requests
from io import BytesIO
import math
from torch.nn.init import xavier_uniform_

In [ ]:
#do not edit this cell
config_url = "https://raw.githubusercontent.com/Arunprakash-A/LLM-from-scratch-PyTorch/main/config_files/dec_config.yml"
response = requests.get(config_url)
config = response.content.decode("utf-8")
config = safe_load(config)
pprint(config)

{'input': {'batch_size': 10, 'embed_dim': 32, 'seq_len': 8, 'vocab_size': 12},
 'model': {'d_ff': 128,
           'd_model': 32,
           'dk': 4,
           'dq': 4,
           'dv': 4,
           'n_heads': 8,
           'n_layers': 6}}


In [ ]:
vocab_size = config['input']['vocab_size']
batch_size = config['input']['batch_size']
seq_len = config['input']['seq_len']
embed_dim = config['input']['embed_dim']
dmodel = embed_dim
dq = torch.tensor(config['model']['dq'])
dk = torch.tensor(config['model']['dk'])
dv = torch.tensor(config['model']['dv'])
heads = torch.tensor(config['model']['n_heads'])
d_ff = config['model']['d_ff']

# Input tokens

* Generate a raw_input ids (without any special tokens appended to it)

* Since we will be using this as label after adding the special  \<start\> token, we use the variable name "label_ids"

* Keep the size of the `label_ids=(bs,seq_len-1)` as we insert a special token ids in the next step

In [ ]:
# do not edit this cell
data_url = 'https://github.com/Arunprakash-A/LLM-from-scratch-PyTorch/raw/main/config_files/w2_input_tokens'
r = requests.get(data_url)
label_ids = torch.load(BytesIO(r.content))

* Let the first token_id be be a special `[start]` token (mapped to integer 0)
* If label_ids=$\begin{bmatrix}1&2\\3&4 \end{bmatrix}$, then we modify it as $\begin{bmatrix}0&1&2\\0&3&4 \end{bmatrix}$

In [ ]:
# the first column of token_ids should be zeros and the rest of the columns come from label_ids
bs, seq_len_minus_one = label_ids.shape
seq_len = seq_len_minus_one + 1  # additional start token

token_ids = torch.zeros(bs, seq_len, dtype = torch.long)
print(token_ids)

print(label_ids) # [BS, seq_len - 1] or [10, 7]
token_ids[:, 1:] = label_ids
print(token_ids) # [10, 8]

tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[ 7,  8,  7,  7,  9,  2,  6],
        [10,  1, 10,  5,  3,  6,  8],
        [ 3,  4,  8,  2, 10, 10, 10],
        [ 4, 10,  1,  3,  4,  9,  7],
        [ 8,  4,  7,  3,  8, 10,  5],
        [ 9,  1,  8,  5,  9,  9, 10],
        [ 7,  3,  8,  2,  5,  1,  5],
        [ 3,  3,  2,  1,  4,  1,  1],
        [10,  9,  9,  9,  6,  9,  2],
        [ 3,  6,  6,  3,  5,  4,  5]])
tensor([[ 0,  7,  8,  7,  7,  9,  2,  6],
        [ 0, 10,  1, 10,  5,  3,  6,  8],
        [ 0,  3,  4,  8,  2, 10, 10, 10],
        [ 0,  4, 10,  1,  3,  4,  9,  7],
        [ 0,  8,  4,  7,  3,  8, 10,  5],
        [ 0,  9,  1,  8,  5,  9,  9, 10],
        [ 0,  7,  3,  8,  

# Implement the following components of a decoder layer

 * Multi-head Masked Attention (MHMA)
 * Multi-head Cross Attention (MHCA)
 * Postion-wise FFN

* Randomly initialize the parameters using normal distribution with the following seed values
  * $W_Q:$(seed=43)
  * $W_K:$(seed=44)
  * $W_V:$(seed=45)
  * $W_O:$(seed=46)

* Remember that, Multi-head cross atention takes two represnetation. One is the encoder output and the other one is the output from masked attetnion sub-layer.

* However, in this assignment, we will fix it to a random matrix.

In [ ]:
class MHCA(nn.Module):

    def __init__(self, dmodel, dq, dk, dv, heads):
        super(MHCA, self).__init__()
        self.heads = heads
        self.dq = dq
        self.dk = dk
        self.dv = dv
        self.dmodel = dmodel
        self.dropout = nn.Dropout(p = 0.1)

        self.WQ = nn.Parameter(self.init_param(dmodel, heads * dq, seed = 43))
        self.WK = nn.Parameter(self.init_param(dmodel, heads * dk, seed = 44))
        self.WV = nn.Parameter(self.init_param(dmodel, heads * dv, seed = 45))
        self.WO = nn.Parameter(self.init_param(heads * dv, dmodel, seed = 46))
        self.scaling_factor = math.sqrt(dk)

    def init_param(self, *size, seed):
        torch.manual_seed(seed)
        return torch.randn(size)

    def forward(self, decoder_input, encoder_output):
        BS, T_dec, _ = decoder_input.shape
        _, T_enc, _ = encoder_output.shape

        Q = torch.matmul(decoder_input, self.WQ.T).view(BS, T_dec, self.heads, -1).transpose(1, 2)
        K = torch.matmul(encoder_output, self.WK.T).view(BS, T_enc, self.heads, -1).transpose(1, 2)
        V = torch.matmul(encoder_output, self.WV.T).view(BS, T_enc, self.heads, -1).transpose(1, 2)

        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scaling_factor
        attention = F.softmax(attention_scores, dim = -1)
        # attention = self.dropout(attention)

        out = torch.matmul(attention, V).transpose(1, 2).contiguous().view(BS, T_dec, -1)
        out = torch.matmul(out, self.WO.T)
        return out

* By default, `mask=None`. Therefore, create and apply the mask while computing the attention scores


In [ ]:
class MHMA(nn.Module):

    def __init__(self, dmodel, dq, dk, dv, heads, mask = None):
        super(MHMA, self).__init__()
        self.heads = heads
        self.dq = dq
        self.dk = dk
        self.dv = dv
        self.dmodel = dmodel
        self.mask = mask
        self.dropout = nn.Dropout(p = 0.1)

        self.WQ = nn.Parameter(self.init_param(heads * dq, dmodel, seed = 43))
        self.WK = nn.Parameter(self.init_param(heads * dk, dmodel, seed = 44))
        self.WV = nn.Parameter(self.init_param(heads * dv, dmodel, seed = 45))
        self.WO = nn.Parameter(self.init_param(dmodel, heads * dv, seed = 46))
        self.scaling_factor = math.sqrt(dk)

    def init_param(self, *size, seed):
        torch.manual_seed(seed)
        return torch.randn(size)

    def forward(self, x, mask=None):
        BS, T, _ = x.shape
        Q = torch.matmul(x, self.WQ.T).view(BS, T, self.heads, -1).transpose(1, 2)
        K = torch.matmul(x, self.WK.T).view(BS, T, self.heads, -1).transpose(1, 2)
        V = torch.matmul(x, self.WV.T).view(BS, T, self.heads, -1).transpose(1, 2)

        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scaling_factor

        if mask is None:
            mask = torch.triu(torch.ones((T, T), device = x.device, dtype = torch.bool), diagonal = 1)
            mask = mask[None, None, :, :].expand(BS, self.heads, T, T)
            attention_scores = attention_scores.masked_fill(mask, float('-inf'))
        else :
            attention_scores = attention_scores + mask

        attention = F.softmax(attention_scores, dim=-1)
        # attention = self.dropout(attention)

        out = torch.matmul(attention, V).transpose(1, 2).contiguous().view(BS, T, -1)
        out = torch.matmul(out, self.WO.T)
        return out

* Implement the FFN and OutputLayer modules (same as the one you implemented for encoder)

In [ ]:
class FFN(nn.Module):

    def __init__(self, dmodel, d_ff):
        super(FFN, self).__init__()
        torch.manual_seed(47)
        self.linear1 = nn.Linear(dmodel, d_ff)
        self.dropout = nn.Dropout(p = 0.1)
        torch.manual_seed(48)
        self.linear2 = nn.Linear(d_ff, dmodel)

        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.kaiming_normal_(self.linear1.weight, mode='fan_out', nonlinearity='relu')
        nn.init.zeros_(self.linear1.bias)
        nn.init.kaiming_normal_(self.linear2.weight, mode='fan_out', nonlinearity='relu')
        nn.init.zeros_(self.linear2.bias)

    def forward(self, x):
        '''
        Input  : Size [BS, T, dmodel]
        Output : Size [BS, T, dmodel]
        '''
        x = self.linear1(x)
        x = F.relu(x)
        # x = self.dropout(x)
        out = self.linear2(x)
        return out

In [ ]:
class OutputLayer(nn.Module):

    def __init__(self, dmodel, vocab_size):
        super(OutputLayer, self).__init__()
        torch.manual_seed(49)
        self.linear = nn.Linear(dmodel, vocab_size)
        self.dropout = nn.Dropout(p = 0.1)
        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.normal_(self.linear.weight, std = 0.02)
        nn.init.zeros_(self.linear.bias)

    def forward(self, representations):
        '''
        Input  :  Size  [BS, T, dmodel]
        Output :  Size  [BS, T, vocab_size]
        Note   : Do not apply the softmax. Just return the output of linear transformation
        '''
        out = self.linear(representations)
        # out = self.dropout(out)
        return out

* Implement the final decoder layer.

In [ ]:
class DecoderLayer(nn.Module):

    def __init__(self, dmodel, dq, dk, dv, d_ff, heads, mask = None):
        super(DecoderLayer, self).__init__()
        self.mhma = MHMA(dmodel, dq, dk, dv, heads, mask=mask)
        self.mhca = MHCA(dmodel, dq, dk, dv, heads)
        self.ffn = FFN(dmodel, d_ff)

        self.pre_ln_mhma = nn.LayerNorm(dmodel)
        self.pre_ln_mhca = nn.LayerNorm(dmodel)
        self.pre_ln_ffn = nn.LayerNorm(dmodel)

    def forward(self, dec_input, enc_output, self_attn_mask = None):
        # Pre-Layer Normalization for all layers
        norm_mhma_in = self.pre_ln_mhma(dec_input)
        mhma_out = self.mhma(norm_mhma_in, self_attn_mask) + dec_input

        norm_mhca_in = self.pre_ln_mhca(mhma_out)
        mhca_out = self.mhca(norm_mhca_in, enc_output) + mhma_out

        norm_ffn_in = self.pre_ln_ffn(mhca_out)
        ffn_out = self.ffn(norm_ffn_in) + mhca_out

        return ffn_out

* Create an embedding layer that takes in token_ids and return embeddings for the token_ids

 * Use seed value: 70

In [ ]:
class Embed(nn.Module):

    def __init__(self, vocab_size, embed_dim):
        super(Embed, self).__init__()
        torch.manual_seed(70)
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.dropout = nn.Dropout(p = 0.1)

    def forward(self, x):
        out = self.embed(x)
        # out = self.dropout(out)
        return out

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len = 500):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        x: Tensor, shape [batch_size, seq_len, embedding_dim]
        """
        x = x + self.pe[:, :x.size(1)]
        return x

# Decoder

 * Implement the decoder that has `num_layers` decoder layers

In [ ]:
class Decoder(nn.Module):

    def __init__(self, vocab_size, dmodel, dq, dk, dv, d_ff, heads, mask, num_layers = 1, max_len = 8):
        super(Decoder, self).__init__()
        self.embed_lookup = Embed(vocab_size, dmodel)
        self.pos_encoder = PositionalEncoding(dmodel, max_len)
        self.dec_layers = nn.ModuleList([DecoderLayer(dmodel, dq, dk, dv, d_ff, heads, mask) for _ in range(num_layers)])
        self.out = OutputLayer(dmodel, vocab_size)

    def forward(self, enc_rep, tar_token_ids):
        tar_embed = self.embed_lookup(tar_token_ids)
        tar_embed = self.pos_encoder(tar_embed)  # Applying positional encodings after the embedding layer
        dec_output = tar_embed

        for layer in self.dec_layers:
            dec_output = layer(dec_output, enc_rep)

        out = self.out(dec_output)
        return out

* Representation from encoder

 * Since all the decoder layers require the representation from the encoder to compute cross attention, we are going to feed in the random values (Note, it does not require gradient during training)

In [ ]:
# do not edit this
enc_rep = torch.randn(size = (batch_size, seq_len, embed_dim), generator = torch.random.manual_seed(10))

# Instantiate the model

In [ ]:
model = Decoder(vocab_size, dmodel, dq, dk, dv, d_ff, heads, mask = None)
print(model)

Decoder(
  (embed_lookup): Embed(
    (embed): Embedding(12, 32)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (pos_encoder): PositionalEncoding()
  (dec_layers): ModuleList(
    (0): DecoderLayer(
      (mhma): MHMA(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (mhca): MHCA(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ffn): FFN(
        (linear1): Linear(in_features=32, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=32, bias=True)
      )
      (pre_ln_mhma): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (pre_ln_mhca): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (pre_ln_ffn): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
  )
  (out): OutputLayer(
    (linear): Linear(in_features=32, out_features=12, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)


In [ ]:
# Print the model's parameters
for name, param in model.named_parameters():
    print(name, param.size())

embed_lookup.embed.weight torch.Size([12, 32])
dec_layers.0.mhma.WQ torch.Size([32, 32])
dec_layers.0.mhma.WK torch.Size([32, 32])
dec_layers.0.mhma.WV torch.Size([32, 32])
dec_layers.0.mhma.WO torch.Size([32, 32])
dec_layers.0.mhca.WQ torch.Size([32, 32])
dec_layers.0.mhca.WK torch.Size([32, 32])
dec_layers.0.mhca.WV torch.Size([32, 32])
dec_layers.0.mhca.WO torch.Size([32, 32])
dec_layers.0.ffn.linear1.weight torch.Size([128, 32])
dec_layers.0.ffn.linear1.bias torch.Size([128])
dec_layers.0.ffn.linear2.weight torch.Size([32, 128])
dec_layers.0.ffn.linear2.bias torch.Size([32])
dec_layers.0.pre_ln_mhma.weight torch.Size([32])
dec_layers.0.pre_ln_mhma.bias torch.Size([32])
dec_layers.0.pre_ln_mhca.weight torch.Size([32])
dec_layers.0.pre_ln_mhca.bias torch.Size([32])
dec_layers.0.pre_ln_ffn.weight torch.Size([32])
dec_layers.0.pre_ln_ffn.bias torch.Size([32])
out.linear.weight torch.Size([12, 32])
out.linear.bias torch.Size([12])


In [ ]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [ ]:
def train(enc_rep, tar_token_ids, label_ids, epochs = 1000):

    loss_trace = []
    for epoch in range(epochs):
        optimizer.zero_grad()

        out = model(enc_rep, tar_token_ids)  # Forward pass
        loss = criterion(out[:, :-1].contiguous().view(-1, out.shape[-1]), label_ids.view(-1))

        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")
        loss_trace.append(loss.item())

* Train the model for 1000 epochs

In [ ]:
train(enc_rep, token_ids, label_ids, 1000)

Epoch 1/1000, Loss: 9.382612228393555
Epoch 2/1000, Loss: 8.747525215148926
Epoch 3/1000, Loss: 7.1064629554748535
Epoch 4/1000, Loss: 6.882545471191406
Epoch 5/1000, Loss: 7.056925296783447
Epoch 6/1000, Loss: 8.491361618041992
Epoch 7/1000, Loss: 4.877208709716797
Epoch 8/1000, Loss: 3.3811452388763428
Epoch 9/1000, Loss: 3.78118634223938
Epoch 10/1000, Loss: 3.2268269062042236
Epoch 11/1000, Loss: 1.8177167177200317
Epoch 12/1000, Loss: 1.4768226146697998
Epoch 13/1000, Loss: 1.858407974243164
Epoch 14/1000, Loss: 1.330363154411316
Epoch 15/1000, Loss: 1.3262995481491089
Epoch 16/1000, Loss: 0.9650627374649048
Epoch 17/1000, Loss: 0.930371880531311
Epoch 18/1000, Loss: 0.4196591377258301
Epoch 19/1000, Loss: 0.10654076933860779
Epoch 20/1000, Loss: 0.05743485316634178
Epoch 21/1000, Loss: 0.03636600077152252
Epoch 22/1000, Loss: 0.017735570669174194
Epoch 23/1000, Loss: 0.010960026644170284
Epoch 24/1000, Loss: 0.009815133176743984
Epoch 25/1000, Loss: 0.00905628316104412
Epoch 26/1

In [ ]:
with torch.inference_mode():
  predictions = torch.argmax(model(enc_rep, token_ids), dim = -1)

* The loss will be around 0.17 after 1000 epochs

In [ ]:
# Number of correct predictions
print(torch.count_nonzero(label_ids == predictions[:, 0:-1]))

tensor(70)


* THe number of correct predictions is close to 66